**Manipulation et analyse des données de la répartition des allocataires percevant au moins une prestation légale selon le quotient familial [EPCI] avec SPARK SQL**

Voici un dashboard Tableau interactif offrant une meilleure visualisation des données : 

[bfc_data.stat.corr("nb_pers_apl", "mt_total_apl")](https://public.tableau.com/app/profile/komlan.augustin.n.guissan/viz/RpartitiondesallocatairespercevantaumoinsuneprestationlgaleselonlequotientfamilialEPCI/DashboardCafData)

In [1]:
# Importation des librairies
import os
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import year

In [2]:
# Création de la session Spark
spark = SparkSession \
    .builder \
    .appName("SparkSQLQueryDataframes") \
    .getOrCreate()

In [3]:
# Chargement des données
data = spark.read.format("csv").load("CAF_cleanedForSQL.csv", header=True, inferSchema= True, sep=",")
data.createOrReplaceTempView("data")
data.show(5)

+-------------------+-------+------------------+----------+----------------+--------------------+-----------------------+---------------------+----------------------+------------------------+----------------------+-----------------------+--------------------------+------------------------+-------------------------+-------------+-----------+------------+------------------------------+----------------------------+-----------------------------+--------------------+
|               date|num_dep|           nom_dep|num_region|      nom_region|         qt_familial|nb_foyers_presta_legale|nb_pers_presta_legale|mt_total_presta_legale|nb_foyers_petite_enfance|nb_pers_petite_enfance|mt_total_petite_enfance|nb_foyers_enfance_jeunesse|nb_pers_enfance_jeunesse|mt_total_enfance_jeunesse|nb_foyers_apl|nb_pers_apl|mt_total_apl|nb_foyers_solidarite_insertion|nb_pers_solidarite_insertion|mt_total_solidarite_insertion|      lieu_residence|
+-------------------+-------+------------------+----------+-------

In [4]:
data.count()

81365

**La base de données contient 81 365 observations**

**EXPLICATION DES VARIABLES**

**date** : *Mois de droit*  
**num_dep** : *Numéro du Département*  
**nom_dep** : *Nom du Département*  
**num_region** : *Numéro de la Région*  
**nom_region** : *Nom de la Région*  
**qt_familial** : *Quotient Familiale*  
**nb_foyers_presta_legale** : *Nombre de Foyers bénéficiares d'au moins une prestation légale selon le quotient familiale*  
**nb_pers_presta_legale** : *Nombre de Personnes appartenant à un foyer bénéficiare d'au moins une prestation légale selon le quotient familiale*  
**mt_total_presta_legale** : *Montant total d'au moins une prestation légale selon le quotient familiale*  
**nb_foyers_petite_enfance** : *Nombre de Foyers bénéficiares d'au moins une prestation petite enfance selon le quotient familiale*  
**nb_pers_petite_enfance** : *Nombre de Personnes appartenant à un foyer bénéficiare d'au moins une prestation petite enfance selon le quotient familiale*  
**mt_total_petite_enfance** : *Montant total d'au moins une prestation petite enfance selon le quotient familiale*  
**nb_foyers_enfance_jeunesse** : *Nombre de Foyers bénéficiares d'au moins une prestation enfance jeunesse selon le quotient familiale*  
**nb_pers_enfance_jeunesse** : *Nombre de Personnes appartenant à un foyer bénéficiare d'au moins une prestation enfance jeunesse selon le quotient familiale*  
**mt_total_enfance_jeunesse** : *Montant total d'au moins une prestation enfance jeunesse selon le quotient familiale*  
**nb_foyers_apl** : *Nombre de Foyers bénéficiares d'au moins une prestation d'aide au logement selon le quotient familiale*  
**nb_pers_apl** : *Nombre de Personnes appartenant à un foyer bénéficiare d'au moins une prestation d'aide au logement selon le quotient familiale*  
**mt_total_apl** : *Montant total d'au moins une prestation d'aide au logement selon le quotient familiale*  
**nb_foyers_solidarite_insertion** : *Nombre de Foyers bénéficiares d'au moins une prestation solidarité insertion selon le quotient familiale*  
**nb_pers_solidarite_insertion** : *Nombre de Personnes appartenant à un foyer bénéficiare d'au moins une prestation solidarité insertion selon le quotient familiale*  
**mt_total_solidarite_insertion** : *Montant total d'au moins une prestation solidarité insertion selon le quotient familiale*  
**lieu_residence** : *Lieu de résidence des bénéficiares des prestations*

In [5]:
#Conversion de la colonne date en format date
data = data.withColumn("date", data["date"].cast("date"))
data.show(5)

+----------+-------+------------------+----------+----------------+--------------------+-----------------------+---------------------+----------------------+------------------------+----------------------+-----------------------+--------------------------+------------------------+-------------------------+-------------+-----------+------------+------------------------------+----------------------------+-----------------------------+--------------------+
|      date|num_dep|           nom_dep|num_region|      nom_region|         qt_familial|nb_foyers_presta_legale|nb_pers_presta_legale|mt_total_presta_legale|nb_foyers_petite_enfance|nb_pers_petite_enfance|mt_total_petite_enfance|nb_foyers_enfance_jeunesse|nb_pers_enfance_jeunesse|mt_total_enfance_jeunesse|nb_foyers_apl|nb_pers_apl|mt_total_apl|nb_foyers_solidarite_insertion|nb_pers_solidarite_insertion|mt_total_solidarite_insertion|      lieu_residence|
+----------+-------+------------------+----------+----------------+-----------------

In [6]:
#ORGANISATION DES DONNEES
data = data.sort("date", "num_dep")
data.show(5)

+----------+-------+-------+----------+--------------------+--------------------+-----------------------+---------------------+----------------------+------------------------+----------------------+-----------------------+--------------------------+------------------------+-------------------------+-------------+-----------+------------+------------------------------+----------------------------+-----------------------------+--------------------+
|      date|num_dep|nom_dep|num_region|          nom_region|         qt_familial|nb_foyers_presta_legale|nb_pers_presta_legale|mt_total_presta_legale|nb_foyers_petite_enfance|nb_pers_petite_enfance|mt_total_petite_enfance|nb_foyers_enfance_jeunesse|nb_pers_enfance_jeunesse|mt_total_enfance_jeunesse|nb_foyers_apl|nb_pers_apl|mt_total_apl|nb_foyers_solidarite_insertion|nb_pers_solidarite_insertion|mt_total_solidarite_insertion|      lieu_residence|
+----------+-------+-------+----------+--------------------+--------------------+-----------------

In [7]:
# MONTANT TOTAL DES DIFFERENTES PRESTATIONS PAR ANNEE
req_viz0 = spark.sql("SELECT year(date) as year, SUM(mt_total_petite_enfance), SUM(mt_total_enfance_jeunesse), SUM(mt_total_solidarite_insertion), SUM(mt_total_apl) \
                     FROM data \
                     GROUP BY year \
                     ORDER BY year DESC")

req_viz0.show()

+----+----------------------------+------------------------------+----------------------------------+-----------------+
|year|sum(mt_total_petite_enfance)|sum(mt_total_enfance_jeunesse)|sum(mt_total_solidarite_insertion)|sum(mt_total_apl)|
+----+----------------------------+------------------------------+----------------------------------+-----------------+
|2024|                  3832752985|                    9379437281|                       10679112603|       7701162044|
|2023|                  7434410649|                   18280014758|                       21909237471|      14995071151|
|2022|                  7320070127|                   16921085541|                       21201964533|      14910022634|
|2021|                  7111168527|                   16267095736|                       21336697540|      15087543167|
|2020|                  7103597785|                   16315363049|                       21724189073|      16082792345|
|2019|                  7553282567|     

In [16]:
# MONTANT TOTAL DES DIFFERENTES PRESTATIONS PAR ANNEE
req_viz0 = spark.sql("SELECT year(date) as year, SUM(mt_total_petite_enfance), SUM(mt_total_enfance_jeunesse), SUM(mt_total_solidarite_insertion), SUM(mt_total_apl), \
                        SUM(mt_total_petite_enfance) + SUM(mt_total_enfance_jeunesse) + SUM(mt_total_solidarite_insertion) + SUM(mt_total_apl) as total, \
                            SUM(mt_total_presta_legale) - total as autres_presta \
                     FROM data \
                     GROUP BY year \
                     ORDER BY year DESC")

req_viz0.show()

+----+----------------------------+------------------------------+----------------------------------+-----------------+-----------+-------------+
|year|sum(mt_total_petite_enfance)|sum(mt_total_enfance_jeunesse)|sum(mt_total_solidarite_insertion)|sum(mt_total_apl)|      total|autres_presta|
+----+----------------------------+------------------------------+----------------------------------+-----------------+-----------+-------------+
|2024|                  3832752985|                    9379437281|                       10679112603|       7701162044|31592464913|   7214287858|
|2023|                  7434410649|                   18280014758|                       21909237471|      14995071151|62618734029|  13533751297|
|2022|                  7320070127|                   16921085541|                       21201964533|      14910022634|60353142835|  12626355704|
|2021|                  7111168527|                   16267095736|                       21336697540|      15087543167|59802

**On observe une différence notable entre le montant total des diverses prestations et celui des prestations légales. Cela pourrait être dû à des arrondis ou bien refléter d'autres prestations non précisées dans la base de données.**

**EXTRACTION ET ANALYSE DES DONNEES DE LA BOURGOGNE FRANCHE-COMTE**

In [25]:
bfc_data = data[data.num_region == 27]
bfc_data.createOrReplaceTempView("bfc_data")

bfc_data.show(5)

+----------+-------+--------------------+----------+--------------------+--------------------+-----------------------+---------------------+----------------------+------------------------+----------------------+-----------------------+--------------------------+------------------------+-------------------------+-------------+-----------+------------+------------------------------+----------------------------+-----------------------------+--------------------+
|      date|num_dep|             nom_dep|num_region|          nom_region|         qt_familial|nb_foyers_presta_legale|nb_pers_presta_legale|mt_total_presta_legale|nb_foyers_petite_enfance|nb_pers_petite_enfance|mt_total_petite_enfance|nb_foyers_enfance_jeunesse|nb_pers_enfance_jeunesse|mt_total_enfance_jeunesse|nb_foyers_apl|nb_pers_apl|mt_total_apl|nb_foyers_solidarite_insertion|nb_pers_solidarite_insertion|mt_total_solidarite_insertion|      lieu_residence|
+----------+-------+--------------------+----------+--------------------

In [31]:
#MONTANT TOTAL DES DIFFERENTES PRESTATIONS EN 2024 REGROUPEES PAR DEPARTEMENT
req_viz1 = spark.sql("SELECT year(date) as year, nom_dep, SUM(mt_total_petite_enfance), SUM(mt_total_enfance_jeunesse), SUM(mt_total_solidarite_insertion), SUM(mt_total_apl) \
                     FROM bfc_data \
                     WHERE year(date) = 2024 \
                     GROUP BY year, nom_dep \
                     ORDER BY nom_dep")

req_viz1.show()

+----+--------------------+----------------------------+------------------------------+----------------------------------+-----------------+
|year|             nom_dep|sum(mt_total_petite_enfance)|sum(mt_total_enfance_jeunesse)|sum(mt_total_solidarite_insertion)|sum(mt_total_apl)|
+----+--------------------+----------------------------+------------------------------+----------------------------------+-----------------+
|2024|           COTE D OR|                    29878495|                      61393120|                          66550185|         58785691|
|2024|               DOUBS|                    29674414|                      70930300|                          74413471|         59558728|
|2024|         HAUTE SAONE|                    12701416|                      28725623|                          27891385|         17498502|
|2024|                JURA|                    12844291|                      29010691|                          25774704|         18082271|
|2024|       

In [36]:
#TEST DE CORRELATION ENTRE LE NOMBRE DE FOYERS BENEFICIAIRES DE L'APL ET LE MONTANT TOTAL DES APL
bfc_data.stat.corr("nb_foyers_apl", "mt_total_apl")

0.9949612394279629

In [37]:
#TEST DE CORRELATION ENTRE LE NOMBRE DE PERSONNES BENEFICIAIRES DE L'APL ET LE MONTANT TOTAL DES APL
bfc_data.stat.corr("nb_pers_apl", "mt_total_apl")

0.9553142138443919

***Comme on pouvait s'y attendre, le nombre de foyers bénéficiaires de l'APL et le montant total des APL sont fortement corrélés.
De même, le nombre de personnes bénéficiaires de l'APL et le montant total des APL sont fortement corrélés.***